# Understanding 6F TCR compared to 6Y

Need to figure out what is wrong with 6F TCR fit results in combination with CAR for predictions. Why is there no antagonism at all?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import json, h5py
import os
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

In [ ]:
from models.tcr_car_akpr_model import (
    activation_function, steady_akpr_i_receptor_types, steady_akpr_i_1ligand, psi_of_i_gamma, 
)
from models.akpr_i_model import psi_of_i
from mcmc.utilities_tcr_tcr_antagonism import check_model_output
from mcmc.mcmc_analysis import find_best_grid_point
from mcmc.prediction_utilities_tcr_car import (
    find_6f_tcr_ampli,
    find_6f_tcr_thresh_fact,
)
from models.akpr_i_model import psi_of_i
from mcmc.plotting import handles_properties_legend
from utils.preprocess import string_to_tuple

from mcmc.costs_tcr_car_antagonism import repackage_tcr_car_params
from mcmc.utilities_tcr_car_antagonism import load_tcr_tcr_akpr_fits

In [ ]:
do_save_plots = False

# Display figures larger
plt.rcParams['figure.dpi'] = 150 # default for me was 75
plt.rcParams["font.family"] = "Arial"# Code taken from 

# https://stackoverflow.com/questions/49223702/adding-a-legend-to-a-matplotlib-plot-with-a-multicolored-line
from matplotlib.legend_handler import HandlerLineCollection
from matplotlib.collections import LineCollection

class HandlerColorLineCollection(HandlerLineCollection):
    def create_artists(self, legend, artist ,xdescent, ydescent,
                        width, height, fontsize,trans):
        x = np.linspace(0,width,self.get_numpoints(legend)+1)
        y = np.zeros(self.get_numpoints(legend)+1)+height/2.-ydescent
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, cmap=artist.cmap,
                     transform=trans)
        lc.set_array(x)
        lc.set_linewidth(artist.get_linewidth())
        return [lc]

# Part 1: single antigen output

## Load 6F and 6Y parameters

In [ ]:
ln10 = np.log(10.0)

In [ ]:
# Load MCMC results for 6F
folder_results = "../results/mcmc/"
with open(os.path.join(folder_results, "mcmc_analysis_tcr_tcr_6f.json"), "r") as h:
    results_6f = json.load(h)
    
f = os.path.join(folder_results, "mcmc_results_tcr_tcr_6f.h5")
samples_file = h5py.File(f, "r")
cost_args_names = samples_file.get("data").attrs.get("cost_args_names")
# ["rates_others", "total_RS", "N", "tau_agonist"]
cost_args_6f = [samples_file.get("data").get(a)[()] for a in cost_args_names]

choice_kmf_6f = string_to_tuple(find_best_grid_point(results_6f, strat="best")[0])  #(1, 3, 1)


# Computed adjustment factor on 6F tau_c threshold and amplitude
# Need to import relevant data
samples_fname = "mcmc_results_tcr_car_both_conc.h5"
results_tcr_car = h5py.File(os.path.join(folder_results, samples_fname), "r")
data_file_name = str(results_tcr_car.get("data").attrs.get("data_file_name"))
df = pd.read_hdf(data_file_name)

# TCR Ag, no CD19 measurements have not been used; also independent data
# to tune 6F and make real predictions.
df_cyto_tcronly = df.xs("None", level="CAR_Antigen")

with open("../data/pep_tau_map_ot1.json", "r") as handle:
    pep_tau_map_ot1 = json.load(handle)

# Adjustment 
#  on tau_c for 6F
res = find_6f_tcr_thresh_fact(df_cyto_tcronly, pep_tau_map_ot1)
tau_c_factor_6f, tau_c_factor_6f_std, tau_cs_fitted = res

# Adjustment of 6F amplitude
ampli_6f = find_6f_tcr_ampli(df_cyto_tcronly)

print("Best (k, m, f) for 6F TCRs:", choice_kmf_6f)
print("6F amplitude:", ampli_6f)
print("6F adjustment factor on tau_c:", tau_c_factor_6f)

In [ ]:
print(tau_cs_fitted)
# This may differ from the MCMC-derived tau_c, because C_N threshold is computed
# from the MCMCed tau_c at maximal L concentration, while this fitted tau_c
# is fitted on data for finite, sub-saturating L. Hence I expect the MCMCed tau_c
# to be smaller, since it is computed at a bigger L. 

In [ ]:
# Import 6Y best fit results
# Load MCMC results for 6F
folder_results = "../results/mcmc/"
with open(os.path.join(folder_results, "mcmc_analysis_akpr_i.json"), "r") as h:
    results_6y = json.load(h)
    
f = os.path.join(folder_results, "mcmc_results_akpr_i.h5")
samples_file = h5py.File(f, "r")
cost_args_names = samples_file.get("data").attrs.get("cost_args_names")
# ["rates_others", "total_RS", "N", "tau_agonist"]
cost_args_6y = [samples_file.get("data").get(a)[()] for a in cost_args_names]

# Best fit is (1, 4, 1) for 6Y TCR and (1, 2, 1) for the CAR part
kmf_6y = find_best_grid_point(results_6y, strat="best")[0]

# Threshold of 6Y TCR fitted on CAR-TCR data
with open(os.path.join(folder_results, "mcmc_analysis_tcr_car_both_conc.json"), "r") as h:
    results_tcr_car = json.load(h)
tau_c_6y = np.exp(results_tcr_car.get("(1, 2, 1)").get("param_estimates").get("MAP best")[4] * ln10)
print("Best (k, m, f) for 6Y TCRs:", kmf_6y)
print(tau_c_6y)

## Check fit results for $k_I = 1$

In [ ]:
def plot_single_ag_output(choice_kmf, results, cost_args, title="Revised AKPR model"):
    pvec = results[str(choice_kmf)]["param_estimates"]["MAP best"]
    pvec = np.exp(np.asarray(pvec)*ln10)

    choice_rates = [pvec[0]] + list(cost_args[0][:1]) + list(pvec[1:3]) + [choice_kmf[0], pvec[3]]
    nmf = (cost_args[2],) + tuple(choice_kmf[1:])
    res = check_model_output(steady_akpr_i_1ligand, choice_rates, cost_args[1], nmf)

    l_range, tau_range, outputs = res
    fig, ax = plt.subplots()
    fig.set_size_inches(5.5, 4.0)
    for i, tau in enumerate(tau_range):
        ax.plot(l_range, outputs[i], label=r"$\tau = {:.0f}$ s".format(tau))
    ax.set(xscale="log", yscale="log", xlabel=r"$L$", ylabel=r"$C_N$", title=title)
    ax.legend(loc="lower right")
    ax.annotate(r"Best $k_I, m, f$ : $({}, {}, {})$".format(*choice_kmf) + "\n"
                + r"Best $\varphi$ : " + "{:.4f}\n".format(pvec[0])
                + r"Best $C_{m, thresh}$ : " + "{:.2f}\n".format(pvec[1])
                + r"Best $I_{thresh}$ : " + "{:.2e}\n".format(pvec[2])
                + r"Best $\psi_{0,TCR}$: " + "{:.2e}\n".format(pvec[3]),
                xy=(0.05, 0.95), ha="left", va="top",
                xycoords="axes fraction")
    fig.tight_layout()
    return [fig, ax]

In [ ]:
fig, ax = plot_single_ag_output(choice_kmf_6f, results_6f, cost_args_6f, 
                                title="Revised AKPR, 6F TCR, single Ag")
if do_save_plots:
    fig.savefig("../figures/model_predictions_widerbounds/revised_akpr_6f_widerbounds_single_antigen_response.pdf", 
               transparent=True, bbox_inches="tight", format="pdf")
plt.show()
plt.close()

# Part 2: TCR/CAR antagonism

## Model parameters
Best TCR/CAR fit is (1, 2, 1) for (k, m, f) of the CAR. 

In [ ]:
with open(os.path.join("..", "data", "pep_tau_map_ot1.json"), "r") as handle:
    pep_tau_map_ot1 = json.load(handle)

In [ ]:
# Load best parameter fit for (1, 2, 1)
fit_conc = ["1uM", "1nM"]
analysis_res_fname = "mcmc_analysis_tcr_car_both_conc.json"
with open(os.path.join("..", "results", "mcmc", analysis_res_fname), "r") as jfile:
    all_results_dicts = json.load(jfile)
    del jfile

# Go back to linear-scale parameters
chosen_kmf = (1, 2, 1)
pvec_best = np.asarray(all_results_dicts.get(str(chosen_kmf)).get("param_estimates").get("MAP best"))

# Load constant parameter values
samples_fname = samples_fname = "mcmc_results_tcr_car_both_conc.h5"
with h5py.File(os.path.join("..", "results", "mcmc", samples_fname), "r") as rfile:
    data_group = rfile.get("data")
    fit_param_names = list(rfile.get("samples").attrs.get("param_names"))
    l_conc_mm_params = data_group.get("l_conc_mm_params")[()]
    cost_args_loaded = [data_group.get(a)[()]
                        for a in data_group.attrs.get("cost_args_names")]
    data_file_name = data_group.attrs.get("data_file_name")
    del data_group, rfile
    
# Print which kmf is the best one
for kmf in all_results_dicts.keys():
    print(kmf, ":", all_results_dicts.get(str(kmf)).get("posterior_probs").get("MAP best"))

In [ ]:
## Load 6F TCR parameters and replace the 6Y default ones with them

# Prepare all data from which correction factors are computed
# keep ITAM numbers levels and TCR Antigen Density
analysis_6f = os.path.join("..", "results", "mcmc", "mcmc_analysis_tcr_tcr_6f.json")
results_6f = os.path.join("..", "results", "mcmc", "mcmc_results_tcr_tcr_6f.h5")

# Load 6F TCR parameters, hope they work with the CAR/TCR params
tcr_6f_loads = load_tcr_tcr_akpr_fits(results_6f, analysis_6f, klim=1)
# params: phi, kappa, cmthresh, I0p, kp, psi0, gamma_tt
# then [N, m, f] of TCR, and I_tot.
tcr_6f_params, tcr_6f_nmf, tcr_6f_itot = tcr_6f_loads
# Replace cost args: ['fixed_rates', 'tcr_car_ritots', 'tcr_car_nmf', 'cd19_tau_l']
# fixed_rates: phi_tcr, kappa_tcr, cmthresh_tcr, ithresh_tcr, k_tcr, psi0_tcr,
# gamma_tt=1.0
cost_args_loaded[0][0:7] = tcr_6f_params
# tcr_car_ritots:  R_tot_tcr, ITp, R_tot_car
cost_args_loaded[1][1] = tcr_6f_itot
# tcr_car_nmf changes to tcr_6f_nmf
cost_args_loaded[2][0:3] = tcr_6f_nmf

# Determine max. output amplitude of 6F relative to 6Y
df = pd.read_hdf(data_file_name)
# This data point, CD19 only and no TCR antigen, has trivially FC=1
# so it's not data we are trying to predict; we use it to set the tau
# threshold of 1-ITAM CAR threshold
df_cyto_cd19only = (df.xs("None", level="TCR_Antigen")
                    .xs("CD19", level="CAR_Antigen"))
# TCR Ag, no CD19 measurements have not been used; also independent data
# to tune 6F and make real predictions.
df_cyto_tcronly = df.xs("None", level="CAR_Antigen")

tcr_ampli_6f, _ = find_6f_tcr_ampli(df_cyto_tcronly)

# Also relative difference in tau threshold based on Hill fits
tcr_thresh_fact, _, _ = find_6f_tcr_thresh_fact(df_cyto_tcronly, pep_tau_map_ot1)
cost_args_loc = list(cost_args_loaded) + [[tcr_ampli_6f, 1.0], tcr_thresh_fact]

In [ ]:
### Rearrange loaded parameters in the correct format 
other_rates, ritot, nmf_fixed, cd19_tau_l, ampli_factors, thresh_factors = cost_args_loc
res = repackage_tcr_car_params(pvec_best, chosen_kmf, other_rates, ritot, nmf_fixed)
(all_rates, tcr_rates, car_rates, ritot_vec, 
    tcr_ri, car_ri, nmf_both, tcr_nmf, car_nmf, threshold_taus) = res

In [ ]:
# Compute thresholds
# Evaluate CAR threshold; TCR is fixed and received as parameter
tcr_thresh = steady_akpr_i_1ligand(tcr_rates, threshold_taus[0]*tcr_thresh_fact,  # TCR threshold factor for 6F
        10*tcr_ri[0], tcr_ri, tcr_nmf, large_l=True)[tcr_nmf[0]]
car_thresh = steady_akpr_i_1ligand(car_rates, threshold_taus[1],
        10*car_ri[0], car_ri, car_nmf, large_l=True)[car_nmf[0]]

# Agonist alone output
ag_alone = steady_akpr_i_1ligand(car_rates, *cd19_tau_l, car_ri, car_nmf)[car_nmf[0]]
ag_alone = activation_function(ag_alone, car_thresh)

In [ ]:
# Print the best fit parameters
for i, p in enumerate(fit_param_names):
    print(p, ":", 10.0**pvec_best[i])
print("TCR rates:", tcr_rates)

## Basic plotting of steady-state solutions
Not forgetting to apply the correction factors on the TCR amplitude and TCR threshold, as we do for the actual model predictions in 6F T cells. 

In [ ]:
# Plot all complexes and all SHP-1s as a function of tau_tcr and L_TCR in presence of CD19. 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range = np.linspace(0.001, 10.0, 101)
l_tcr_range = np.asarray([10, 100, 1000, 10000, 100000])

model_columns = pd.Index([r"$T_{}$".format(n) for n in range(tcr_nmf[0]+1)] 
                   + [r"$C_{}$".format(n) for n in range(car_nmf[0]+1)]
                   + [r"$I^T$", r"$I^C$", r"$Z^T$", r"$Z^C$", "Ratio"], name="Variable")
model_index = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L^T$", r"$\tau^T$"])
df_model = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                       columns=model_columns, index=model_index)

for l_tcr, tau_tcr in model_index:
    taus = np.asarray([tau_tcr, cd19_tau_l[0]])
    lvec = np.asarray([l_tcr, cd19_tau_l[1]])
    complexes_mix = steady_akpr_i_receptor_types(all_rates, taus, lvec, ritot_vec, nmf_both)
    df_model.loc[(l_tcr, tau_tcr), r"$T_0$":r"$T_{}$".format(tcr_nmf[0])] = complexes_mix[0]
    df_model.loc[(l_tcr, tau_tcr), r"$C_0$":r"$C_{}$".format(car_nmf[0])] = complexes_mix[1]
    df_model.loc[(l_tcr, tau_tcr), r"$I^T$":r"$I^C$"] = complexes_mix[2]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model.loc[(l_tcr, tau_tcr), r"$Z^T$"] = tcr_ampli_6f*activation_function(complexes_mix[0][-1], tcr_thresh)
    df_model.loc[(l_tcr, tau_tcr), r"$Z^C$"] = activation_function(complexes_mix[1][-1], car_thresh)
    df_model.loc[(l_tcr, tau_tcr), "Ratio"] = (df_model.loc[(l_tcr, tau_tcr), r"$Z^T$":r"$Z^C$"].sum() 
                                                    / ag_alone)

In [ ]:
for k in df_model.index:
    psis = psi_of_i_gamma(df_model.loc[k, r"$I^T$":r"$I^C$"].values, 
                      all_rates[3], all_rates[4], all_rates[0], all_rates[5], all_rates[6])
    df_model.loc[k, r"$\psi^T$"] = psis[0]
    df_model.loc[k, r"$\psi^C$"] = psis[1]

In [ ]:
df_model.loc[:, r"$I^T$":r"$I^C$"].sum(axis=1).max()

In [ ]:
df_plot = df_model + df_model.max(axis=0)*1e-4
df_plot = df_plot.loc[:, r"$I^T$":]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L^T$"] = df_plot[r"$L^T$"].astype(str)

g = sns.relplot(data=df_plot, x=r"$\tau^T$", y=0, col="Variable", hue=r"$L^T$", 
               height=2.0, col_wrap=4, kind="line", facet_kws=dict(sharey=False), 
               hue_order=l_tcr_range[::-1].astype(str), palette="magma")
for ax in g.axes.flat:
    ax.set_yscale("log")
plt.show()
plt.close()

## Output for the TCR alone – details

In [ ]:
# Plot all complexes and all SHP-1s as a function of L_TCR for a few tau_TCR
#tau_tcr_range2 = np.sort(np.asarray(list(pep_tau_map.values())))
#l_tcr_range2 = np.logspace(1.0, 5.0, 101)

model_columns_tcr = pd.Index([r"$C^T_{}$".format(n) for n in range(tcr_nmf[0]+1)]
                   + [r"$I$", r"$Z^T$"], name="Variable")
model_index_tcr = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L^T$", r"$\tau^T$"])
df_model_tcr = pd.DataFrame(np.zeros([len(model_index_tcr), len(model_columns_tcr)]), 
                       columns=model_columns_tcr, index=model_index_tcr)

tcr_rates_arr = [np.asarray([a]) for a in tcr_rates]
tcr_rates_arr.insert(5, np.ones(1))
tcr_ri_arr = [np.asarray([tcr_ri[0]]), tcr_ri[1]]
tcr_nmf_arr = [np.asarray([a]) for a in tcr_nmf]
for l_tcr, tau_tcr in model_index_tcr:
    complexes = steady_akpr_i_1ligand(tcr_rates, tau_tcr, l_tcr, tcr_ri, tcr_nmf)
    tau_arr = np.asarray([tau_tcr])
    l_arr = np.asarray([l_tcr])
    
    # Compare to receptors types output for a single type. Should be the same, assert to be sure. 
    complexes_mix = steady_akpr_i_receptor_types(tcr_rates_arr, tau_arr, l_arr, tcr_ri_arr, tcr_nmf_arr)
    #assert np.allclose(complexes[:-1] - complexes_mix[0], 0.0), str(complexes[:-1]) + " " + str(complexes_mix[0])
    #assert np.allclose(complexes[-1] - complexes_mix[1], 0.0)

    df_model_tcr.loc[(l_tcr, tau_tcr), r"$C^T_0$":r"$C^T_{}$".format(tcr_nmf[0])] = complexes[:-1]
    df_model_tcr.loc[(l_tcr, tau_tcr), r"$I$"] = complexes[-1]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model_tcr.loc[(l_tcr, tau_tcr), r"$Z^T$"] = tcr_ampli_6f*activation_function(complexes[-2], tcr_thresh)

In [ ]:
for k in df_model_tcr.index:
    psi = psi_of_i(df_model_tcr.loc[k, r"$I$"], tcr_rates[3], tcr_rates[4], tcr_rates[0], tcr_rates[5])
    df_model_tcr.loc[k, r"$\psi$"] = psi

## Plots to explain the effect

Plot 1 : how $\psi_T$ behaves as a function of $\tau$ with TCR alone. 

Plot 2: how $\psi_T$ and $\psi_C$ behave as a function of $\tau$, for a couple of $L$. How that explains the ratio as a superposition of $T_N$ and $C_N$. 

Plot 3: how they would behave if $\gamma_{TC} = 1$. 

In [ ]:
df_plot = df_model_tcr + df_model_tcr.max(axis=0)*1e-4
ytitles = [r"$C^T_2$", r"$I$", r"$\psi$", r"$C^T_4$"]
df_plot = df_plot.loc[:, ytitles]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L^T$"] = df_plot[r"$L^T$"].astype(str)
g = sns.relplot(data=df_plot, x=r"$\tau^T$", y=0, col="Variable", hue=r"$L^T$", 
            height=2.0, col_wrap=len(ytitles), kind="line", facet_kws=dict(sharey=False), 
            hue_order=l_tcr_range[::-1].astype(str), palette="viridis")
for i, ax in enumerate(g.axes.flat):
    ax.set_yscale("log")
    ax.set_ylabel(ytitles[i])
g.tight_layout()
plt.show()
plt.close()

In [ ]:
df_plot = df_model + df_model.max(axis=0)*1e-4
ytitles = [r"$I^T$", r"$\psi^C$", r"$I^C$", r"$\psi^T$", r"$Z^C$", r"$Z^T$", "Ratio"]
df_plot = df_plot.loc[:, ytitles]
df_plot = df_plot.loc[:, r"$I^T$":]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L^T$"] = df_plot[r"$L^T$"].astype(str)

g = sns.relplot(data=df_plot, x=r"$\tau^T$", y=0, col="Variable", hue=r"$L^T$", 
               height=2.0, col_wrap=4, kind="line", facet_kws=dict(sharey=False), 
               hue_order=l_tcr_range[::-1].astype(str), palette="magma")
for i, ax in enumerate(g.axes.flat):
    ax.set_yscale("log")
    ax.set_ylabel(ytitles[i])
g.tight_layout()
plt.show()
plt.close()

In [ ]:
# Plot S_T, S_C on the same graph, to show tradeoff. Maybe plot psi_c on a second axis? 
# Plot Z_T, Z_C, and their total on the same graph. For two Ls. Maybe normalize by Z_C for CAR alone. 
choice_l = [100000, 10000]
palette = sns.color_palette(n_colors=len(choice_l))
palette = {choice_l[i]:palette[i] for i in range(len(choice_l))}

fig, axes = plt.subplots(3)
fig.set_size_inches(5, 3*3)
axes = axes.flatten()

# Plot the S variables
ax = axes[0]
styles = {r"$I^T$":("-", None), r"$I^C$":("--", None)}#, r"$S_T + S_C$":("-", None)}  # for T, C
ylow = np.inf
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$I^C$"]
    if y_c.min() < ylow:
        ylow = y_c.min()
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$I^T$"]
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$I^T$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$I^C$"][0])
    #ax.plot(tau_tcr_range, y_t + y_c, color=palette[choice_l[i]], linestyle=styles[r"$S_T + S_C$"][0])

# Custom legend
hues = (r"$L^T$", palette)
styles = ("Receptor type", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ylims = ax.get_ylim()
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"Active $I$", ylim=(ylow*0.75, ylims[1]))

# Plot the psi variables
ax = axes[1]
styles = {r"$\psi^T$":("-", None), r"$\psi^C$":("--", None)}
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$\psi^C$"]
    y_c = y_c / y_c.iloc[0]
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$\psi^T$"]
    y_t = y_t / y_t.iloc[0]
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$\psi^C$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$\psi^T$"][0])
    
# Custom legend
hues = (r"$L^T$", palette)
styles = ("Receptor type", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"$\psi\, /\, \psi(\tau^T = 0)$")


# Plot the Z variables
ax = axes[2]
styles = {r"$Z^T$":("--", None), r"$Z^C$":(":", None), r"$Z_{tot}$":("-", None)}  # for T, C, and total

ylow = np.inf
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$Z^C$"]
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$Z^T$"]
    if y_c.min() < ylow:
        ylow = y_c.min()
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$Z^T$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$Z^C$"][0])
    ax.plot(tau_tcr_range, y_t + y_c, color=palette[choice_l[i]], linestyle=styles[r"$Z_{tot}$"][0])

# Custom legend
hues = (r"$L^T$", palette)
styles = ("Output", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ylims = ax.get_ylim()
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"Normalized output $Z$", ylim=(ylow*0.75, ylims[1]))

fig.tight_layout()
plt.show()
plt.close()

### As a function of $L_T$ for a couple different $\tau_T$
Show agonist, partial agonist, antagonist

In [ ]:
# Plot all complexes and all SHP-1s as a function of tau_tcr and L_TCR in presence of CD19. 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range = np.asarray([2.0, 4.0, 10.0])
l_tcr_range = np.logspace(0, 5, 201)

model_columns = pd.Index([r"$C^T_{}$".format(n) for n in range(tcr_nmf[0]+1)] 
                   + [r"$C^C_{}$".format(n) for n in range(car_nmf[0]+1)]
                   + [r"$I^T$", r"$I^C$", r"$Z^T$", r"$Z^C$", "Ratio"], name="Variable")
model_index = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L^T$", r"$\tau^T$"])
df_model = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                       columns=model_columns, index=model_index)

for l_tcr, tau_tcr in model_index:
    taus = np.asarray([tau_tcr, cd19_tau_l[0]])
    lvec = np.asarray([l_tcr, cd19_tau_l[1]])
    complexes_mix = steady_akpr_i_receptor_types(all_rates, taus, lvec, ritot_vec, nmf_both)
    df_model.loc[(l_tcr, tau_tcr), r"$C^T_0$":r"$C^T_{}$".format(tcr_nmf[0])] = complexes_mix[0]
    df_model.loc[(l_tcr, tau_tcr), r"$C^C_0$":r"$C^C_{}$".format(car_nmf[0])] = complexes_mix[1]
    df_model.loc[(l_tcr, tau_tcr), r"$I^T$":r"$I^C$"] = complexes_mix[2]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model.loc[(l_tcr, tau_tcr), r"$Z^T$"] = tcr_ampli_6f*activation_function(complexes_mix[0][-1], tcr_thresh)
    df_model.loc[(l_tcr, tau_tcr), r"$Z^C$"] = activation_function(complexes_mix[1][-1], car_thresh)
    df_model.loc[(l_tcr, tau_tcr), "Ratio"] = (df_model.loc[(l_tcr, tau_tcr), r"$Z^T$":r"$Z^C$"].sum() 
                                                    / ag_alone)

In [ ]:
def strip_names(df, char, axis=0):
    stripper = lambda x: x.strip(char)
    return df.rename(mapper=stripper, axis=axis)

def pad_names(df, char, axis=0):
    padder = lambda x: char + x + char
    return df.rename(mapper=padder, axis=axis)

In [ ]:
# Two subplots, one for each TCR tau. Shared legend. 
# New version with 4 mini-cartoons. Two plots on top of each other. 
tcr_ags_palette = ["#4c78bbff", "#d83ad8ff", "#DA3833"]  # antagonist, partial agonist, agonist
car_cd19_color = "grey"
labelsize = 8
antagonism_palette = np.asarray(sns.color_palette('PuOr_r', n_colors=100))[[13, 87]]  # Antagonism, enhancement
print(sns.color_palette(antagonism_palette).as_hex()[:])
palette = list(sns.color_palette("Greys", n_colors=2))
fig, axes = plt.subplots(3, 1)
axes = axes.flatten()
fig.set_size_inches(3.0, 1.8*2)
for i, tcr_tau in enumerate(tau_tcr_range):
    ax = axes[i]
    y_c = df_model.loc[(l_tcr_range, tcr_tau), r"$Z^C$"]
    y_t = df_model.loc[(l_tcr_range, tcr_tau), r"$Z^T$"]
    y_tot = (y_c + y_t) / ag_alone
    y_c = y_c / ag_alone
    y_t = y_t / ag_alone
    
    # plot the same data on both axes
    # Code from https://stackoverflow.com/questions/38051922/how-to-get-differents
    # -colors-in-a-single-line-in-a-matplotlib-figure
    # select how to color
    cmap = mpl.colors.ListedColormap(antagonism_palette)
    norm = mpl.colors.BoundaryNorm([-np.inf, 1.0], cmap.N)
    # get segments
    xy = np.array([l_tcr_range, y_tot]).T.reshape(-1, 1, 2)
    segments = np.hstack([xy[:-1], xy[1:]])
    # make line collection
    lc = mpl.collections.LineCollection(segments, cmap=cmap, norm=norm, 
                                        label="Total", lw=2.5)
    lc.set_array(y_tot)
    ax.add_collection(lc)
    ligand_type = "antagonist" if i == 0 else "partial agonist"
    ax.plot(l_tcr_range, y_c, label=r"$Z^C$", color=palette[1], ls=":", lw=2.0)
    ax.plot(l_tcr_range, y_t, label=r"$Z^T$", color=tcr_ags_palette[i], ls="--", lw=2.0)
    
    #ax.set_ylim(-0.1, df_model["Ratio"].max()*1.1)
    ax.set_xlabel("TCR Ag density (#)", size=labelsize, labelpad=0.1)
    ax.set_xscale("log")
    ax.axhline(1.0, ls='-', color="k", lw=0.75)
    ax.set_ylabel(r"$Z(\mathrm{mix})\,/\,Z^C(\mathrm{CD19})$", size=labelsize, labelpad=0.2)
    # Create final legend inside the first plot
    if i == 1:
        # Change order of legend
        handles, labels = ax.get_legend_handles_labels()
        labels.insert(0, labels.pop())
        handles.insert(0, handles.pop())
        # Change color of the Z_T line before drawing legend
        back_color = handles[2].get_color()
        handles[2].set_color(handles[1].get_color())

        ax.legend(handles=handles, labels=labels, 
                  handler_map={lc: HandlerColorLineCollection(numpoints=2)},
                  loc="center right", bbox_to_anchor=(-0.15, 0.4), frameon=False,
                  #frameon=True, edgecolor=(1, 1, 1, 0), facecolor=(1, 1, 1, 0.9), framealpha=0.9, 
                  #labelspacing=0.2, handletextpad=0.4, columnspacing=0.5,  borderaxespad=0.3, borderpad=0.2, 
                  fontsize=labelsize, title="Output", title_fontsize=labelsize)
        # Restor line color on the plot
        handles[2].set_color(back_color)
    
    # Force more log ticks
    locmaj = mpl.ticker.LogLocator(base=10,numticks=5) 
    ax.xaxis.set_major_locator(locmaj)
    # Force minor log ticks
    locmin = mpl.ticker.LogLocator(base=10.0,subs=np.arange(0.1, 1.0, 0.1), numticks=5)
    ax.xaxis.set_minor_locator(locmin)
    ax.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())
    # Change label size
    ax.tick_params(axis='both', which='major', labelsize=labelsize*0.85, pad=0.1, length=3.0)
    ax.tick_params(axis='both', which='minor', pad=0.1, length=2.0)
    
    # Hide unnecessary spines to make room for diagrams. 
    for s in ["top", "right"]:
        ax.spines[s].set_visible(False)

fig.tight_layout(w_pad=0.3, h_pad=0.3)
if do_save_plots:
    fig.savefig("figures/model_details/akpr_i_output_contributions_6F_TCR.pdf", 
                bbox_inches="tight", transparent=True)
plt.show()
plt.close()